In [1]:
import sys
from pathlib import Path
wdir = Path.cwd().parent.absolute()
sys.path.insert(1, str(wdir))

from models.EEGModels import EEGNet, ShallowConvNet, DeepConvNet
import torch
import os

In [2]:
model = EEGNet(nb_classes = 5, Chans = 14, Samples = 128, dropoutRate = 0.1, kernLength = 16, F1 = 64, D = 8, F2 = 64, dropoutType = 'Dropout')

total_params = sum(p.numel() for p in model.parameters())
print("Total number of parameters: ", total_params)

Total number of parameters:  51717


In [3]:
print(os.cpu_count())
print(torch.cuda.device_count()) 
device = torch.device('cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu')
print(device)
print(device.type == 'cuda')
if device.type == 'cuda':
    properties = torch.cuda.get_device_properties(device)
    print(properties)

8
0
mps
False


In [5]:
import subprocess
import plistlib

def get_gpu_info():
    """Get information about the GPU on macOS."""
    # Run system_profiler command and capture the output
    result = subprocess.run(['system_profiler', '-xml', 'SPDisplaysDataType'], capture_output=True)
    if result.returncode == 0:
        # Parse the XML output
        display_info = plistlib.loads(result.stdout)
        # Extract GPU information
        gpu_count = len(display_info[0]['_items'])
        return gpu_count
    else:
        print("Error:", result.stderr)
        return None

gpu_count = get_gpu_info()
if gpu_count is not None:
    print("Number of GPUs:", gpu_count)


[{'_name': 'Apple M2', 'spdisplays_mtlgpufamilysupport': 'spdisplays_metal3', 'spdisplays_ndrvs': [{'_name': 'Color LCD', '_spdisplays_display-product-id': 'a052', '_spdisplays_display-serial-number': 'fd626d62', '_spdisplays_display-vendor-id': '610', '_spdisplays_display-week': '0', '_spdisplays_display-year': '0', '_spdisplays_displayID': '1', '_spdisplays_pixels': '2940 x 1912', '_spdisplays_resolution': '1470 x 956 @ 60.00Hz', 'spdisplays_ambient_brightness': 'spdisplays_no', 'spdisplays_connection_type': 'spdisplays_internal', 'spdisplays_display_type': 'spdisplays_built-in-liquid-retina', 'spdisplays_main': 'spdisplays_yes', 'spdisplays_mirror': 'spdisplays_off', 'spdisplays_online': 'spdisplays_yes', 'spdisplays_pixelresolution': 'spdisplays_2560x1664Retina'}, {'_name': 'HP Z27n', '_spdisplays_display-product-id': '3219', '_spdisplays_display-serial-number': '1010101', '_spdisplays_display-vendor-id': '22f0', '_spdisplays_display-week': '16', '_spdisplays_display-year': '2016',

In [21]:
import time
import torch

start_time = time.time()

# syncrocnize time with cpu, otherwise only time for oflaoding data to gpu would be measured
torch.mps.synchronize()

a = torch.ones(4000,4000, device="mps")
for _ in range(200):
   a +=a

elapsed_time = time.time() - start_time
print( "GPU Time: ", elapsed_time)

GPU Time:  0.3293938636779785
